In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader

class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // reduction, in_channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

In [2]:
def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            layers += [SEBlock(v)]  # Add SE Block after each conv layer
            in_channels = v
    return nn.Sequential(*layers)

cfgs = {
    'VGG16_SE': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
}

class VGG_SE(nn.Module):
    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG_SE, self).__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

# Instantiate the model
vgg16_se = VGG_SE(make_layers(cfgs['VGG16_SE']))

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [8]:
# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets and create dataloaders (as you've defined)
train_dataset = datasets.ImageFolder(root='../dataset/training', transform=transform)
test_dataset = datasets.ImageFolder(root='../dataset/testing', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [9]:
vgg16_se = vgg16_se.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16_se.parameters(), lr=0.01, momentum=0.9)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    vgg16_se.train()
    running_loss = 0.0
    running_corrects = 0
    total_batches = len(train_loader)
    
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = vgg16_se(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{total_batches}], Loss: {loss.item():.4f}')


    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}')

print('Finished Training')

Epoch [1/5], Batch [1/878], Loss: 6.7186
Epoch [1/5], Batch [2/878], Loss: 6.7221
Epoch [1/5], Batch [3/878], Loss: 6.6799
Epoch [1/5], Batch [4/878], Loss: 6.6901
Epoch [1/5], Batch [5/878], Loss: 6.6564
Epoch [1/5], Batch [6/878], Loss: 6.6480
Epoch [1/5], Batch [7/878], Loss: 6.6333
Epoch [1/5], Batch [8/878], Loss: 6.5633
Epoch [1/5], Batch [9/878], Loss: 6.5239
Epoch [1/5], Batch [10/878], Loss: 6.4791
Epoch [1/5], Batch [11/878], Loss: 6.4514
Epoch [1/5], Batch [12/878], Loss: 6.3740
Epoch [1/5], Batch [13/878], Loss: 6.2992
Epoch [1/5], Batch [14/878], Loss: 6.2398
Epoch [1/5], Batch [15/878], Loss: 6.1118
Epoch [1/5], Batch [16/878], Loss: 6.1657
Epoch [1/5], Batch [17/878], Loss: 6.0826
Epoch [1/5], Batch [18/878], Loss: 6.0132
Epoch [1/5], Batch [19/878], Loss: 6.0455
Epoch [1/5], Batch [20/878], Loss: 5.8754
Epoch [1/5], Batch [21/878], Loss: 5.8242
Epoch [1/5], Batch [22/878], Loss: 5.5797
Epoch [1/5], Batch [23/878], Loss: 5.4262
Epoch [1/5], Batch [24/878], Loss: 5.4909
E

C:\Users\Carmi\OneDrive\Documents\GitHub\PhotoLingo\venv\Lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/5], Batch [131/878], Loss: 0.7870
Epoch [1/5], Batch [132/878], Loss: 0.9512
Epoch [1/5], Batch [133/878], Loss: 0.8639
Epoch [1/5], Batch [134/878], Loss: 1.0993
Epoch [1/5], Batch [135/878], Loss: 0.7206
Epoch [1/5], Batch [136/878], Loss: 1.1954
Epoch [1/5], Batch [137/878], Loss: 0.8498
Epoch [1/5], Batch [138/878], Loss: 0.9178
Epoch [1/5], Batch [139/878], Loss: 1.0914
Epoch [1/5], Batch [140/878], Loss: 1.0518
Epoch [1/5], Batch [141/878], Loss: 0.9433
Epoch [1/5], Batch [142/878], Loss: 1.1400
Epoch [1/5], Batch [143/878], Loss: 0.8777
Epoch [1/5], Batch [144/878], Loss: 0.9654
Epoch [1/5], Batch [145/878], Loss: 0.9463
Epoch [1/5], Batch [146/878], Loss: 0.8933
Epoch [1/5], Batch [147/878], Loss: 0.9501
Epoch [1/5], Batch [148/878], Loss: 0.9719
Epoch [1/5], Batch [149/878], Loss: 0.8512
Epoch [1/5], Batch [150/878], Loss: 0.8771
Epoch [1/5], Batch [151/878], Loss: 1.0750
Epoch [1/5], Batch [152/878], Loss: 0.6968
Epoch [1/5], Batch [153/878], Loss: 1.0334
Epoch [1/5]

In [10]:
# Save the entire model
torch.save(vgg16_se, '../models/vgg16_se_full_model.pth')

In [ ]:
import torch
import numpy as np
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

model = vgg16_se
 
model.eval()  # Set model to evaluation mode
y_pred = []
y_true = []

# No gradients needed for evaluation
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        y_pred.extend(predicted.cpu().numpy())  # Append batch predictions
        y_true.extend(labels.cpu().numpy())  # Append true labels

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Calculate precision, recall, F1-score and support for each class
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average=None)

# Calculate overall accuracy
accuracy = np.sum(np.diag(conf_matrix)) / np.sum(conf_matrix)


print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Accuracy: {accuracy}%')
print(f'Precision (per class): {precision}')
print(f'Recall (per class): {recall}')
print(f'F1 Score (per class): {f1_score}')

C:\Users\Carmi\OneDrive\Documents\GitHub\PhotoLingo\venv\Lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
